In [25]:
import numpy as np
from sklearn import preprocessing 
import tensorflow as tf
import pandas as pd

In [26]:
data_preprocessed = pd.read_csv('data_preprocessed_test_assign.csv')
data_preprocessed

,Unnamed: 0,Age,Location 1,Location 2,Location_3,Location_4,Location_5,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,Churn
0,0,63,0,0,1,0,0,17,73.36,236,0
1,1,62,0,0,0,0,1,1,48.76,172,0
2,2,24,0,0,1,0,0,5,85.47,460,0
3,3,36,0,0,0,1,0,3,97.94,297,1
4,4,46,0,0,0,1,0,19,58.14,266,0
...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,33,0,1,0,0,0,23,55.13,226,1
99996,99996,62,0,0,0,0,1,19,61.65,351,0
99997,99997,64,1,0,0,0,0,17,96.11,251,1
99998,99998,51,0,0,0,0,1,20,49.25,434,1


In [27]:
targets = np.where(data_preprocessed['Churn'], 1, 0)

In [28]:
targets.sum()/targets.shape[0]

0.49779

In [29]:
unscaled_inputs = data_preprocessed.drop(['Churn'],axis=1)

In [30]:
unscaled_inputs

,Unnamed: 0,Age,Location 1,Location 2,Location_3,Location_4,Location_5,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB
0,0,63,0,0,1,0,0,17,73.36,236
1,1,62,0,0,0,0,1,1,48.76,172
2,2,24,0,0,1,0,0,5,85.47,460
3,3,36,0,0,0,1,0,3,97.94,297
4,4,46,0,0,0,1,0,19,58.14,266
...,...,...,...,...,...,...,...,...,...,...
99995,99995,33,0,1,0,0,0,23,55.13,226
99996,99996,62,0,0,0,0,1,19,61.65,351
99997,99997,64,1,0,0,0,0,17,96.11,251
99998,99998,51,0,0,0,0,1,20,49.25,434


In [31]:
from sklearn.preprocessing import StandardScaler

assignment_scaler = StandardScaler()

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns):
        self.scaler = StandardScaler()
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [33]:
unscaled_inputs = unscaled_inputs.drop(['Unnamed: 0'], axis=1)

In [34]:
columns_to_omit = ['Location 1', 'Location 2', 'Location_3',
       'Location_4','Location_5']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]
assignment_scaler = CustomScaler(columns_to_scale)

In [35]:
assignment_scaler.fit(unscaled_inputs)

C:\ProgramData\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


CustomScaler(columns=['Age', 'Subscription_Length_Months', 'Monthly_Bill',
                      'Total_Usage_GB'])

In [36]:
scaled_inputs = assignment_scaler.transform(unscaled_inputs)

In [37]:
scaled_inputs

,Age,Location 1,Location 2,Location_3,Location_4,Location_5,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB
0,1.241670,0,0,1,0,0,0.651115,0.410606,-0.294289
1,1.176226,0,0,0,0,1,-1.658879,-0.805374,-0.784852
2,-1.310651,0,0,1,0,0,-1.081380,1.009204,1.422681
3,-0.525321,0,0,0,1,0,-1.370129,1.625597,0.173279
4,0.129120,0,0,0,1,0,0.939864,-0.341720,-0.064338
...,...,...,...,...,...,...,...,...,...
99995,-0.721654,0,1,0,0,0,1.517363,-0.490504,-0.370939
99996,1.176226,0,0,0,0,1,0.939864,-0.168220,0.587191
99997,1.307115,1,0,0,0,0,0.651115,1.535140,-0.179313
99998,0.456341,0,0,0,0,1,1.084239,-0.781153,1.223390


In [38]:
scaled_inputs.shape

(100000, 9)

In [39]:
from sklearn.model_selection import train_test_split 
train_test_split(scaled_inputs, targets)

[            Age  Location 1  Location 2  Location_3  Location_4  Location_5  \
 13854  1.634335           0           0           0           1           0   
 42550  1.110782           0           0           1           0           0   
 19702  1.241670           0           1           0           0           0   
 98860  0.129120           0           0           1           0           0   
 5428  -1.637872           0           0           0           0           1   
 ...         ...         ...         ...         ...         ...         ...   
 90739  0.521785           1           0           0           0           0   
 29533 -1.114319           0           0           1           0           0   
 91826 -1.572428           0           0           0           0           1   
 19933  0.979894           0           1           0           0           0   
 82120 -0.198101           0           0           0           1           0   
 
        Subscription_Length_Months  Mo

In [40]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [42]:
reg = LogisticRegression()
reg.fit(x_train,y_train)

LogisticRegression()

In [43]:
reg.score(x_train,y_train)

0.5049625

In [44]:
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 0, ..., 1, 0, 0])

In [45]:
from sklearn.metrics import precision_recall_fscore_support

In [46]:
#preci
precision_recall_fscore_support(model_outputs,y_train)

(array([0.62577833, 0.38323714]),
 array([0.50550269, 0.50407631]),
 array([0.55924677, 0.43542846]),
 array([49703, 30297], dtype=int64))

In [47]:
#deploying the model 
import pickle
with open('modelA', 'wb') as file:
    pickle.dump(reg, file)

In [48]:
with open('scalerA','wb') as file:
    pickle.dump(assignment_scaler, file)